In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



/Users/marijnkoolen/Code/Huygens/republic-project


In [2]:
from republic.elastic.republic_elasticsearch import initialize_es
from republic.config.republic_config import set_config_inventory_num

inv_num = 3760
inv_config = set_config_inventory_num(inv_num, ocr_type="pagexml")

rep_es = initialize_es(host_type="external")

rep_es

In [6]:
import republic.extraction.extract_resolution_metadata as extract_res

# initialise a set of searchers that have all the proposition phrases
# 1. a FuzzyPhraseSearcher that has all the known phrases related to propositions
# 2. a FuzzyTemplateSearcher that has some known proposition opening templates that
#    tries to find phrases that together match the template
# 3. a VariableMatcher that identifies the text between phrase matches in the template
#    and labels these in-between texts as part of the proposer or location.
prop_searchers = extract_res.generate_proposition_searchers()

for searcher_type in prop_searchers:
    print(searcher_type, prop_searchers[searcher_type], '\n')

phrase <fuzzy_search.fuzzy_phrase_searcher.FuzzyPhraseSearcher object at 0x11471a800> 

template {'proposition_from_correspondence': FuzzyTemplateSearcher(template=FuzzyTemplate(labels={'proposition_verb', 'representation_relation', 'location', 'resolution_relation', 'person_location', 'proposer_name', 'residence_relation', 'person_role', 'title', 'addressee_name', 'correspondence_from', 'affairs_relation', 'temporal_reference', 'organisation', 'addressed_to', 'person_name_prefix', 'proposition_opening', 'location_relation'}, required=['proposition_opening'])), 'proposition_from_statement': FuzzyTemplateSearcher(template=FuzzyTemplate(labels={'proposition_verb', 'resident_in', 'representation_relation', 'location', 'resolution_relation', 'proposer_name', 'residence_relation', 'person_role', 'title', 'addressee_name', 'affairs_relation', 'temporal_reference', 'organisation', 'addressed_to', 'person_name_prefix', 'proposition_opening', 'location_relation'}, required=['proposition_opening

In [7]:
# Fetch some resolutions for testing

resolutions = []
for ri, res in enumerate(rep_es.scroll_inventory_resolutions(inv_num)):
    resolutions.append(res)
    if len(resolutions) == 10:
        break

total hits: {'value': 7088, 'relation': 'eq'} 	hits per scroll: 10


In [8]:
import json

# The skip formulas are proposition opening formulas for which we don't know 
# the structure of the full opening sentence, so there is no fitting template.

skip_formulas = {
    'heeft aan haar Hoog Mog. voorgedragen',
    'heeft ter Vergadering gecommuniceert ',
}

for ri, resolution in enumerate(resolutions):
    if len(resolution.evidence) == 0:
        print('resolution without evidence:', resolution.metadata)
    # Fetch all phrase matches for the resolution.
    phrase_matches = extract_res.get_paragraph_phrase_matches(rep_es, resolution)
    # create a copy of the resolution with extended metadata, using the template
    # and variable searchers.
    new_resolution = extract_res.add_resolution_metadata(resolution, phrase_matches,
                                                         prop_searchers['template'],
                                                         prop_searchers['variable'])
    if 'proposition_type' not in new_resolution.metadata or new_resolution.metadata['proposition_type'] is None:
        new_resolution.metadata['proposition_type'] = 'unknown'
    print(json.dumps(new_resolution.metadata, indent=4))


/opt/homebrew/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in search requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


{
    "inventory_num": 3760,
    "source_id": "session-1705-07-06-num-1",
    "type": "resolution",
    "id": "session-1705-07-06-num-1-resolution-7",
    "session_date": "1705-07-06",
    "session_id": "session-1705-07-06-num-1",
    "session_num": 1,
    "president": null,
    "session_year": 1705,
    "session_month": 7,
    "session_day": 6,
    "session_weekday": "Lunae",
    "proposition_type": "missive",
    "proposer": {
        "title": {
            "text": "den Secretaris",
            "start_offset": 26,
            "end_offset": 40
        },
        "variable_entity": {
            "text": "Mortaigne ,",
            "start_offset": 40,
            "end_offset": 51
        },
        "full_string": {
            "text": "den Secretasis Mortaigne ",
            "start_offset": 26,
            "end_offset": 51
        }
    },
    "decision": null,
    "resolution_type": "ordinaris",
    "text_page_num": [
        850
    ],
    "index_timestamp": "2022-05-23T13:47:34.040724

{
    "inventory_num": 3760,
    "source_id": "session-1705-07-06-num-1",
    "type": "resolution",
    "id": "session-1705-07-06-num-1-resolution-11",
    "session_date": "1705-07-06",
    "session_id": "session-1705-07-06-num-1",
    "session_num": 1,
    "president": null,
    "session_year": 1705,
    "session_month": 7,
    "session_day": 6,
    "session_weekday": "Lunae",
    "proposition_type": "missive",
    "proposer": null,
    "decision": null,
    "resolution_type": "ordinaris",
    "text_page_num": [
        850
    ],
    "index_timestamp": "2022-05-23T13:47:35.036878",
    "formula": {
        "proposition_opening": {
            "text": "ONtfangen een Missive van",
            "start_offset": 0,
            "end_offset": 25
        },
        "full_string": {
            "text": "Ontfangen een Missive van",
            "start_offset": 0,
            "end_offset": 25
        }
    },
    "proposition_origin": "Londen",
    "proposition_verb": {
        "proposition_verb"